<a href="https://colab.research.google.com/github/pedrov718/Content-Based-Book-Rec-System/blob/main/book_reccomendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy.cli
spacy.cli.download("en_core_web_lg")

import re
import string
import nltk
import numpy as np

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import data_table
data_table.enable_dataframe_formatter()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs.csv')

In [ ]:
df.head()

In [ ]:
df['Blurb'][10:15]

In [ ]:
# Load spacy
nlp = spacy.load('en_core_web_lg')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    #Remove stop words
    text = text.split()

    #Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', word) for word in text]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string


In [ ]:
print("Cleaning train data...\n")

df["blurb_clean"] = df['Blurb'].apply(lambda x: clean_string(x, stem='Spacy'))

In [ ]:
df.head(2)

In [ ]:
df['Title_lower'] = df['Title'].apply(lambda x: x.lower())

In [ ]:
df['blurb_clean_title'] =  df['Title_lower'] +  ' ' + df['blurb_clean']

In [ ]:
df['blurb_clean_title'][10]

In [ ]:
df.to_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs_clean.csv')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run locally here

In [13]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Book_Rec/books_with_blurbs_clean.csv')

#### Now that we have the data cleaned and saved to Drive we can use tfidf to convert our text data to numerical

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['blurb_clean_title'])

In [11]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index])

In [14]:
most_similiar_book()

Write a line from your favorite book? hello

 Title: 
 Exegesis (Vintage Contemporaries) 

 Description: 
 -----------,Hello, Alice.,------------------------------------------------------------------,Date: Sun, 16 Jan 2000 14:27:39 (PST),From: edgar@cyprus.stanford.edu,To: Alice@cs.stanford.edu,Subject: Hello ,------------------------------------------------------------------,Hello, Alice.,      


#### TOP N Number of book reccomendations

In [15]:
def top_n_similiar_books():
  
  n = int(input("How many books would you like reccomended? Please input a number. "))

  sentence = [input("Write the title or line from your favorite book? ")]
  
  y = vectorizer.transform(sentence)
  
  cosine_sim = cosine_similarity(X,y)

  # Get the pairwsie similarity scores
  sim_scores = list(enumerate(cosine_sim))
  # Sort the books based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # Get the scores for n most similar books
  sim_scores = sim_scores[0:n]
  
  titles = []
  author = []
  desc = []

  for pair in sim_scores:
    indx, score = pair
    titles.append(df["Title"][indx])
    author.append(df["Author"][indx])
    desc.append(df["Blurb"][indx])
  
  reccs = pd.DataFrame(list(zip(titles, author, desc)), columns =['titles', 'author', 'blurbs'])
  
  return reccs

In [16]:
top_n_similiar_books()

How many books would you like reccomended? Please input a number. 2
Write the title or line from your favorite book? Crows


,titles,author,blurbs
0,"Crows and Jays: A Guide to the Crows, Jays and...",Steve Madge,"London, 1676: when spy-turned-playwright Aphra..."
1,Still Life With Crows,Lincoln Child,A small Kansas town has turned into a killing...


In [17]:
!pip install git+https://github.com/internetarchive/openlibrary-client.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/internetarchive/openlibrary-client.git to /tmp/pip-req-build-x4lhydpf
  Running command git clone --filter=blob:none --quiet https://github.com/internetarchive/openlibrary-client.git /tmp/pip-req-build-x4lhydpf
  Resolved https://github.com/internetarchive/openlibrary-client.git to commit 61ce9703c10ef06cef745ff50915a9e86ddd5451
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel 

In [18]:
from olclient.openlibrary import OpenLibrary
ol = OpenLibrary()
author_olid = ol.Author.get_olid_by_name('Dan Brown')
author_obj = ol.get(author_olid)

In [19]:
from IPython.display import display, HTML

In [20]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  author_olid = ol.Author.get_olid_by_name(df['Author'].iloc[max_index])
  if ol.get(author_olid) != None:
    author_obj = ol.get(author_olid)
  else:
    print('Sorry author is not in our database')
  if author_obj.bio != None:
    author_desc = author_obj.bio.split('\r\n\r\n')[0]
  else:
    author_desc = f"Sorry we dont have any information about {df['Author'].iloc[max_index]} at the moment"
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index], '\n\n', 'About the Author:', '\n', author_desc, 
               display(HTML('<img src="https://covers.openlibrary.org/b/isbn/' + df['ISBN'].iloc[max_index] + '-L.jpg" />')))

In [21]:
most_similiar_book()

Write a line from your favorite book? crow



 Title: 
 Seven Crows  (Buffy the Vampire Slayer and Angel) 

 Description: 
 "One crow sorrow, Two crows mirth,,Three crows a wedding,,Four crows a birth,,Five crows silver,,Six crows gold,,Seven crows a secret,,Which must never be told....",In a sleepy little town on the border between Arizona and Mexico, Agent Riley Finn and his operative wife, Sam, have tracked down an international smuggling ring involving vampires. Surprisingly the call for reinforcements is answered by Buffy Summers and the atoning vampire Angel.,Now tempers are flaring in the heat of the day -- and night -- as people are dying and locals are turning a blind eye to the deadly events. Bodies are turning up in the surrounding desert, some drained of blood, some having succumbed to another, fast-moving death. Riley Finn is noticing the arrival of more and more crows to this area, ominous portents of the events ahead. But even Mr. Secret Agent Man is distracted from his job when his wife goes undercover with Angel.